In [44]:
#Importing libraries
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam, RMSprop , SGD, Adadelta
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, Input
from tensorflow.keras import Sequential
from tensorflow.keras import Model
import PIL
from tensorflow.keras.preprocessing.image import ImageDataGenerator #allows us to augment and process data fed into a CNN
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os,cv2,random
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator #A tool that allows us to augment and process data to be fed into a CNN
from sklearn.model_selection import KFold
import numpy as np

In [45]:
# Model configuration
batch_size = 50
img_size, img_num_channels = 121, 3
loss_function = categorical_crossentropy
no_classes = 4
no_epochs = 20
optimizer1 = Adam()
optimizer2 = SGD
optimizer3 = RMSprop
optimizer4 = Adadelta
verbosity = 1

In [46]:
data_path='D:/amitavsir/bcd/datacompressed/train'
list_folder=os.listdir(path = data_path)
data=[]   
for i in list_folder:
    new_path=os.path.join(data_path,i) 
    pic_list=os.listdir(new_path)                                               
    for img in pic_list:
        pic=os.path.join(new_path,img)   
        arr=cv2.imread(pic)    
        data.append([arr,list_folder.index(i)])    
        
random.shuffle(data)  
x_train,y_train=[],[]
for i,j in data:
    x_train.append(i)
    y_train.append(j)
x_train=np.array(x_train).reshape(-1,img_size,img_size,img_num_channels)
y_train=np.array(y_train).reshape(-1,1)

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train)
X_train = x_train/255
y_train = y_train.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X_train,y_train, test_size=0.2)

In [47]:
# Parse numbers as floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [48]:
# Define per-fold score containers <-- these are new
acc_per_fold = [] #stores accuracy
loss_per_fold = [] #stores losses

In [49]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

In [50]:
#class for training set
train_data = ImageDataGenerator(rescale = 1./255, #pixel values rescaled so that it stays between 0 and 1.
                                shear_range = 0.2, #this and the next 4 steps are used to augment our data by shearing it, flipping and zooming it to produce more examples per image.
                                zoom_range = 0.2,  #Data augmentation allows us to prevent overfitting of data into the training set.
                                horizontal_flip = True,
                                vertical_flip = True,
                                rotation_range = 40,
                                width_shift_range = 0.2,
                                height_shift_range = 0.2,
                                fill_mode = 'nearest'
                                )
test_data = ImageDataGenerator(rescale = 1/.255)

train_set = train_data.flow_from_directory('D:/amitavsir/bcd/datacompressed/train', #Image path
                                           target_size = (img_size,img_size), #The shape which we want to input our images in our model
                                           batch_size = 20,         #The batch size in mini batch gradient descent
                                           class_mode = 'categorical')   


test_set = test_data.flow_from_directory('D:/amitavsir/bcd/datacompressed/validation',
                                         target_size = (img_size,img_size),
                                         batch_size = 50,
                                         class_mode = 'categorical')

Found 962 images belonging to 4 classes.
Found 500 images belonging to 4 classes.


In [67]:
num_of_train_samples=962
num_of_validation_samples=500

In [54]:
#model = tf.keras.Sequential()
model.add(Input(shape=(img_size, img_size, 3)))
model.add(Conv2D(128, (5,5), strides=(2,2), activation="relu"))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(rate = 0.1))
model.add(Conv2D(64, 3, activation="relu"))
model.add(MaxPooling2D(3))
model.add(Dense(50, activation='relu'))
model.add(Conv2D(32, 3, activation="relu"))
model.add(Dropout(rate = 0.15))
model.add(Conv2D(32, 3, activation="relu"))
model.add(MaxPooling2D(3))
model.add(Conv2D(32, 3, activation="relu"))
model.add(MaxPooling2D(2))
model.add(Dense(4, activation='softmax'))
model.summary()

ValueError: Input 0 of layer conv2d_14 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 1028]

In [ ]:
model.fit_generator(train_set, 
                    steps_per_epoch = int(962/20)
                    epochs=20, 
                    verbose=1,
                    validation_data = test_set,
                    validation_steps = 5,
                    callbacks=model_callbacks)

In [85]:
from tensorflow.keras.layers import Activation,MaxPooling2D
# Build model
model = Sequential()
model.add(Conv2D(128, (1, 1), input_shape=(img_size, img_size, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (1, 1), padding='valid'))
#model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dense(128))
model.add(Conv2D(256, (1, 1), padding='valid'))
model.add(Activation('relu'))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (1, 1), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

In [86]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [87]:
#Train
model.fit_generator(train_set, 
                    steps_per_epoch = int(962/30),
                    epochs=30, 
                    verbose=1,
                    validation_data = test_set,
                    validation_steps = 5)

Epoch 1/30
32/32 [==============================] - 23s 707ms/step - loss: 1.3953 - acc: 0.2672 - val_loss: 298.9569 - val_acc: 0.2520
Epoch 2/30
32/32 [==============================] - 20s 635ms/step - loss: 1.3874 - acc: 0.2588 - val_loss: 39.5789 - val_acc: 0.2480
Epoch 3/30
32/32 [==============================] - 21s 649ms/step - loss: 1.3883 - acc: 0.2299 - val_loss: 12.0750 - val_acc: 0.2680
Epoch 4/30
32/32 [==============================] - 21s 653ms/step - loss: 1.3863 - acc: 0.2749 - val_loss: 19.3141 - val_acc: 0.2520
Epoch 5/30
32/32 [==============================] - 22s 672ms/step - loss: 1.3853 - acc: 0.2547 - val_loss: 66.4647 - val_acc: 0.2520
Epoch 6/30
32/32 [==============================] - 21s 662ms/step - loss: 1.3836 - acc: 0.2669 - val_loss: 46.7293 - val_acc: 0.2480
Epoch 7/30
32/32 [==============================] - 22s 686ms/step - loss: 1.3879 - acc: 0.2875 - val_loss: 78.8636 - val_acc: 0.2520
Epoch 8/30
32/32 [==============================] - 21s 660ms